# MeanHamilMinimizer_rigetti_scipy
* Feedback loop between Qubiter and Rigetti QVM
* minimization via scipy.optimize.minimize

>This notebook calls Rigetti's method QVMConnection() which only works if you first:
* install the Rigetti Forest SDK available at https://www.rigetti.com/forest
* open a second terminal (besides the one that runs this notebook) and type "qvm -S" in it
* open a third terminal and type "quilc -S" in it

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/rrtucci/PycharmProjects/qubiter/qubiter/jupyter_notebooks
/home/rrtucci/PycharmProjects/qubiter


In [2]:
from qubiter.adv_applications.MeanHamil_native import *
from qubiter.adv_applications.MeanHamil_rigetti import *
from qubiter.adv_applications.MeanHamilMinimizer import *

loaded OneQubitGate, WITHOUT autograd.numpy


In [3]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil import get_qc

In [4]:
qvm_url = "http://localhost:5000"
compiler_server_address = "tcp://localhost:5555"
forest_url = "https://forest-server.qcs.rigetti.com"
qvm = QVMConnection(endpoint=qvm_url, compiler_endpoint=compiler_server_address)

## First Example (taken from Pennylane docs). 

In [5]:
qc = get_qc('2q-qvm')

In [6]:
num_qbits = 2
file_prefix = 'mean_hamil_rigetti_test1'
emb = CktEmbedder(num_qbits, num_qbits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(0, rads='#1')
wr.write_Ry(0, rads='#2')
wr.close_files()

In [7]:
wr.print_eng_file(jup=True)

1,ROTX #1 AT 0,2,ROTY #2 AT 0


In [8]:
wr.print_pic_file(jup=True)

1,| Rx,2,| Ry


In [9]:
fun_name_to_fun = None

In [10]:
hamil = QubitOperator('Z0', 1.)
print('hamil=\n', hamil)

hamil=
 1.0 [Z0]


In [11]:
init_var_num_to_rads = {1: .3, 2: .8}
all_var_nums = [1, 2]

In [12]:
num_samples = 100
rand_seed = 1234
print_hiatus = 10
verbose = False

In [13]:
num_samples = 100
print_hiatus = 10
verbose = False
np.random.seed(1234)

In [14]:
emp_mhamil = MeanHamil_rigetti(qc, file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, num_samples=num_samples)
targ_mhamil = MeanHamil_native(file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, simulator_name='SEO_simulator') # zero samples

In [15]:
mini = MeanHamilMinimizer(emp_mhamil, targ_mhamil,
                 all_var_nums, init_var_num_to_rads,
                 print_hiatus=print_hiatus, verbose=verbose)

In [16]:
emp_mhamil.translator.print_aqasm_file()


pg += RX(rads1*(-2), 0)
pg += RY(rads2*(-2), 0)




In [17]:
print(emp_mhamil.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[2]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RX(rads1*-2) 0
RY(rads2*-2) 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [18]:
mini.find_min(minlib='scipy', method='Powell')

x_val~ (#1, #2)
iter=0, cost=-0.020000, x_val=0.300000, 0.800000
iter=10, cost=0.100000, x_val=-0.609830, 0.800000
iter=20, cost=-0.360000, x_val=-0.575388, 1.418034
iter=30, cost=-0.620000, x_val=-2.867184, 4.381854
iter=40, cost=0.920000, x_val=-3.260178, 2.996423
iter=50, cost=-0.940000, x_val=-1.645093, 2.996423
iter=60, cost=-0.980000, x_val=-1.642198, 2.996423
iter=70, cost=-0.960000, x_val=-1.642141, 2.996423
iter=80, cost=-0.560000, x_val=-2.049606, 3.457236
iter=90, cost=-0.880000, x_val=-1.630944, 2.983760


   direc: array([[ 1.        ,  0.        ],
       [-1.06675607,  1.20642308]])
     fun: -0.98
 message: 'Optimization terminated successfully.'
    nfev: 95
     nit: 2
  status: 0
 success: True
       x: array([-1.63027098,  2.98299858])

## Second, more complicated example

In [19]:
qc = get_qc('4q-qvm')

In [20]:
num_qbits = 4
file_prefix = 'mean_hamil_rigetti_test2'
emb = CktEmbedder(num_qbits, num_qbits)
wr = SEO_writer(file_prefix, emb)
wr.write_Ry(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2*.5')
# wr.write_Rn(3, rads_list=['#1', '-#1*3', '#2'])
wr.write_Rx(1, rads='-#1*.3')
wr.write_cnot(2, 3)
wr.write_qbit_swap(1, 2)
wr.close_files()

In [21]:
wr.print_eng_file(jup=True)

1,ROTY 25.714286 AT 2,2,ROTX #2*.5 AT 1,3,ROTX -#1*.3 AT 1,4,SIGX AT 3 IF 2T,5,SWAP 2 1


In [22]:
wr.print_pic_file(jup=True)

1,| Ry | |,2,| | Rx |,3,| | Rx |,4,X---@ | |,5,| <---> |


In [23]:
fun_name_to_fun = None

In [24]:
hamil = QubitOperator('X1 Y3 X1 Y1', .4) + QubitOperator('Y2 X1', .7)
print('hamil=\n', hamil)

hamil=
 0.7 [X1 Y2] +
0.4 [Y1 Y3]


In [25]:
init_var_num_to_rads = {1: 2.1, 2: 3.4}
all_var_nums = [1, 2]

In [26]:
num_samples = 100
print_hiatus = 25
verbose = False
np.random.seed(1234)

In [27]:
emp_mhamil = MeanHamil_rigetti(qc, file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, num_samples=num_samples)
targ_mhamil = MeanHamil_native(file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, simulator_name='SEO_simulator') # zero samples

In [28]:
mini = MeanHamilMinimizer(emp_mhamil, targ_mhamil,
                 all_var_nums, init_var_num_to_rads,
                 print_hiatus=print_hiatus, verbose=verbose)

In [29]:
emp_mhamil.translator.print_aqasm_file()


pg += RY(-0.8975979109989651, 2)
pg += RX(rads2*.5*(-2), 1)
pg += RX(-rads1*.3*(-2), 1)
pg += CNOT(2, 3)
pg += SWAP(2, 1)




In [30]:
print(emp_mhamil.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[4]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RY(-0.8975979109989651) 2
RX(rads2*0.5*-2) 1
RX(-1*rads1*0.3*-2) 1
CNOT 2 3
SWAP 2 1
RY(-pi/2) 1
RX(pi/2) 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]
MEASURE 3 ro[3]



In [31]:
mini.find_min(minlib='scipy', method='Powell')

x_val~ (#1, #2)
iter=0, cost=0.340000, x_val=2.100000, 3.400000
iter=25, cost=0.340000, x_val=3.969473, 3.537767
iter=50, cost=0.232000, x_val=5.014259, 2.549414


   direc: array([[1., 0.],
       [0., 1.]])
     fun: 0.23199999999999998
 message: 'Optimization terminated successfully.'
    nfev: 59
     nit: 2
  status: 0
 success: True
       x: array([5.01425901, 2.54941381])